In [1]:
from pathlib import Path
import pandas as pd
import matplotlib as plt
import json
import numpy as np
import scipy.stats as stats
import hvplot.pandas
import requests

# Google maps query


In [14]:
%%writefile googlemaps.py
# Create a for loop to imput lat and lng to grab locations


import requests

def get_chicago_neighborhood(lat, lon, api_key):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    location = f"{lat},{lon}"
    params = {
        "latlng": location,
        "result_type": "neighborhood",
        "key": api_key,
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    print(data)
    print(response)
    if data["status"] == "OK":
        # Extract neighborhood information from the first result
        neighborhood = None
        for result in data["results"]:
            for address_component in result["address_components"]:
                if "neighborhood" in address_component["types"]:
                    neighborhood = address_component["long_name"]
                    break
            if neighborhood:
                break
        return neighborhood

    else:
        return None

# Replace with your Google Maps API Key
api_key = "API_Key_HERE"
lat = 41.8781  # Example latitude for Chicago
lon = -87.6298  # Example longitude for Chicago

neighborhood = get_chicago_neighborhood(lat, lon, api_key)
if neighborhood:
    print(f"Neighborhood: {neighborhood}")
else:
    print("Neighborhood not found.")

Writing googlemaps.py


In [25]:
income_df = Path("Resources/income_by_Neighborhood.csv")
dfi = pd.set_option('display.max_colwidth', 1)
dfi = pd.read_csv(income_df)

dfi

,Neighborhood,Median household income (dollars)
0,Albany Park,"$40,711.00"
1,Archer Heights,"$39,431.00"
2,Armour Square,"$22,756.00"
3,Ashburn,"$53,633.00"
4,Auburn Gresham,"$34,238.00"
...,...,...
68,Washington Park,"$15,160.00"
69,West Elsdon,"$45,310.00"
70,West Englewood,"$26,693.00"
71,West Garfield Park,"$23,121.00"


In [32]:
#Remove 
dfi["Median household income (dollars)"]=dfi["Median household income (dollars)"].str.replace("$", "")
dfi["Median household income (dollars)"]=dfi["Median household income (dollars)"].str.replace(",", "")
dfi["Median household income (dollars)"]=dfi["Median household income (dollars)"].astype("float64")
dfi

,Neighborhood,Median household income (dollars)
0,Albany Park,40711.0
1,Archer Heights,39431.0
2,Armour Square,22756.0
3,Ashburn,53633.0
4,Auburn Gresham,34238.0
...,...,...
68,Washington Park,15160.0
69,West Elsdon,45310.0
70,West Englewood,26693.0
71,West Garfield Park,23121.0


In [33]:
dfi.dtypes

Neighborhood                         object 
Median household income (dollars)    float64
dtype: object

In [17]:
crimes_df = Path("Resources/Crimes_-_2022.csv")
dfc = pd.read_csv(crimes_df, encoding="UTF-8")
dfc

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12589893,JF109865,01/11/2022 03:00:00 PM,087XX S KINGSTON AVE,1565,SEX OFFENSE,INDECENT SOLICITATION OF A CHILD,RESIDENCE,False,True,...,7.0,46,17,1194660.0,1847481.0,2022,09/14/2023 03:41:59 PM,41.736409,-87.562410,"(41.736409029, -87.562410309)"
1,12592454,JF113025,01/14/2022 03:55:00 PM,067XX S MORGAN ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,RESIDENCE,False,True,...,16.0,68,26,1170805.0,1860170.0,2022,09/14/2023 03:41:59 PM,41.771782,-87.649437,"(41.771782439, -87.649436929)"
2,12785595,JF346553,08/05/2022 09:00:00 PM,072XX S UNIVERSITY AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,APARTMENT,True,False,...,5.0,69,17,1185135.0,1857211.0,2022,09/14/2023 03:41:59 PM,41.763338,-87.597001,"(41.763337967, -87.597001131)"
3,12808281,JF373517,08/14/2022 02:00:00 PM,055XX W ARDMORE AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,...,39.0,11,17,1138383.0,1937953.0,2022,09/14/2023 03:41:59 PM,41.985875,-87.766404,"(41.985875279, -87.766403857)"
4,12888104,JF469015,11/10/2022 03:47:00 AM,072XX S MAY ST,1477,WEAPONS VIOLATION,RECKLESS FIREARM DISCHARGE,STREET,False,False,...,17.0,68,15,1169903.0,1856822.0,2022,09/14/2023 03:41:59 PM,41.762615,-87.652840,"(41.76261474, -87.652840463)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238990,27038,JF387886,09/08/2022 10:50:00 AM,061XX N WINTHROP AVE,0110,HOMICIDE,FIRST DEGREE MURDER,PORCH,True,False,...,48.0,77,01A,1167747.0,1940929.0,2022,12/29/2022 03:40:26 PM,41.993457,-87.658318,"(41.993457327, -87.658318065)"
238991,26819,JF275138,06/10/2022 11:41:00 PM,018XX S CANAL ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,25.0,31,01A,1173423.0,1891309.0,2022,09/19/2022 03:41:05 PM,41.857173,-87.638918,"(41.857173304, -87.638917814)"
238992,26601,JF132803,02/03/2022 04:27:00 PM,000XX E 100TH PL,0110,HOMICIDE,FIRST DEGREE MURDER,AUTO,False,False,...,9.0,49,01A,1178641.0,1838358.0,2022,09/19/2022 03:41:05 PM,41.711753,-87.621374,"(41.711753121, -87.621374343)"
238993,26857,JF290575,06/23/2022 02:25:00 AM,027XX S KEDVALE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,22.0,30,01A,1149162.0,1885416.0,2022,09/01/2022 03:42:17 PM,41.841506,-87.728122,"(41.841506072, -87.728122297)"


In [24]:
len(dfc[dfc["Community Area"].isin([1, 2, 3])])

11874

In [18]:
public_health = Path("Resources/Public_Health_Services-_Chicago_Primary_Care_Community_Health_Centers.csv")
dfh = pd.read_csv(public_health, encoding="UTF-8")
dfh

,Facility,Community Area (#),Phone,"FQHC, Look-alike, or Neither; Special Notes",Address
0,Mercy Family Health Center @ Oakwood Shores,OAKLAND (36),(773) 451-0460,Look-alike,"3753 S. Cottage Grove60653\n(41.826722480000456, -87.60828727799964)"
1,ACCESS Southwest Family Health Center,GARFIELD RIDGE (56),(866) 882-2237,FQHC,"4839 W. 47th Street60638\n(41.80757461200045, -87.74460191199967)"
2,Heartland Health Outreach- Refugee Health,UPTOWN (3),(773) 751-1744,FQHC; specialize in refugee health,"4750 N Sheridan Rd60640\n(41.96843068300046, -87.65485692899966)"
3,Heartland Health Center- Hibbard Elementary School,ALBANY PARK (14),(773) 336-6100,FQHC; School-based health center (open to community),"4930 North Sawyer Avenue60625\n(41.97084703000047, -87.70977434799966)"
4,Near North - Winfield Moody Health Center,NEAR NORTH SIDE (8),(312) 337-1073,FQHC,"1276 N. Clybourn60610\n(41.90535451700049, -87.64178597999967)"
...,...,...,...,...,...
115,Mercy Family Health Center @ Mercy Hospital,NEAR SOUTH SIDE (33),(312) 567-2000,Look-alike,"2525 S. Michigan60616\n(41.846811956000465, -87.62352337499965)"
116,ACCESS Cabrini Family Health Center,MCKINLEY PARK (59),(866) 882-2237,FQHC,"3450 S. Archer Ave.60608\n(41.8312221810005, -87.67633950099969)"
117,Alivio Medical Center -Little Village Lawndale High School Campus,SOUTH LAWNDALE (30),(773) 254-1400,FQHC,"3120 S Kostner Ave60623\n(41.83605293800048, -87.73406839999967)"
118,UIC - Mile Square - Englewood,ENGLEWOOD (68),(312) 996-2000,FQHC,"641 W. 63rd Street60621\n(41.77979320600048, -87.64095807999968)"


In [19]:
dfh.head()

,Facility,Community Area (#),Phone,"FQHC, Look-alike, or Neither; Special Notes",Address
0,Mercy Family Health Center @ Oakwood Shores,OAKLAND (36),(773) 451-0460,Look-alike,"3753 S. Cottage Grove60653\n(41.826722480000456, -87.60828727799964)"
1,ACCESS Southwest Family Health Center,GARFIELD RIDGE (56),(866) 882-2237,FQHC,"4839 W. 47th Street60638\n(41.80757461200045, -87.74460191199967)"
2,Heartland Health Outreach- Refugee Health,UPTOWN (3),(773) 751-1744,FQHC; specialize in refugee health,"4750 N Sheridan Rd60640\n(41.96843068300046, -87.65485692899966)"
3,Heartland Health Center- Hibbard Elementary School,ALBANY PARK (14),(773) 336-6100,FQHC; School-based health center (open to community),"4930 North Sawyer Avenue60625\n(41.97084703000047, -87.70977434799966)"
4,Near North - Winfield Moody Health Center,NEAR NORTH SIDE (8),(312) 337-1073,FQHC,"1276 N. Clybourn60610\n(41.90535451700049, -87.64178597999967)"


In [20]:
to_drop = ["Phone", "FQHC, Look-alike, or Neither; Special Notes"]
dfh = dfh.drop(columns=to_drop)

In [21]:
dfh

,Facility,Community Area (#),Address
0,Mercy Family Health Center @ Oakwood Shores,OAKLAND (36),"3753 S. Cottage Grove60653\n(41.826722480000456, -87.60828727799964)"
1,ACCESS Southwest Family Health Center,GARFIELD RIDGE (56),"4839 W. 47th Street60638\n(41.80757461200045, -87.74460191199967)"
2,Heartland Health Outreach- Refugee Health,UPTOWN (3),"4750 N Sheridan Rd60640\n(41.96843068300046, -87.65485692899966)"
3,Heartland Health Center- Hibbard Elementary School,ALBANY PARK (14),"4930 North Sawyer Avenue60625\n(41.97084703000047, -87.70977434799966)"
4,Near North - Winfield Moody Health Center,NEAR NORTH SIDE (8),"1276 N. Clybourn60610\n(41.90535451700049, -87.64178597999967)"
...,...,...,...
115,Mercy Family Health Center @ Mercy Hospital,NEAR SOUTH SIDE (33),"2525 S. Michigan60616\n(41.846811956000465, -87.62352337499965)"
116,ACCESS Cabrini Family Health Center,MCKINLEY PARK (59),"3450 S. Archer Ave.60608\n(41.8312221810005, -87.67633950099969)"
117,Alivio Medical Center -Little Village Lawndale High School Campus,SOUTH LAWNDALE (30),"3120 S Kostner Ave60623\n(41.83605293800048, -87.73406839999967)"
118,UIC - Mile Square - Englewood,ENGLEWOOD (68),"641 W. 63rd Street60621\n(41.77979320600048, -87.64095807999968)"
